In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/pSp_PrivacyEncoder')

Mounted at /content/drive


In [2]:
import numpy as np

from PIL import Image
from tqdm import tqdm

import torch
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms

from torch.utils.data import DataLoader, Dataset


In [3]:
#original_path = "image_datasets/test_images"
#processed_path = "image_datasets/test_inferenced_privacy0.3"
#processed_path = "image_datasets/test_blurred"
#processed_path = "image_datasets/test_mosaiced"
##############################################################################
BATCH_SIZE = 4
NUM_WORKERS = 4

IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP', '.tiff'
]


In [4]:
def is_image_file(filename):
  return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)
  
class LoadDataset(Dataset):
  def __init__(self, path, transforms=None):
    files_path = []
    assert os.path.isdir(path), '{} is not a valid directory'.format(path)
    for root, dirs, files in sorted(os.walk(path)):
      for file_name in files:
        if is_image_file(file_name):
          file_path = os.path.join(root, file_name)
          files_path.append(file_path)
    
    files_path = sorted(files_path)
    self.files = files_path
    self.transforms = transforms

  def __len__(self):
    return len(self.files)

  def __getitem__(self, idx):
    img_path = self.files[idx]
    img = Image.open(img_path).convert('RGB')

    if self.transforms:
      img = self.transforms(img)
    return img


In [5]:
def MSE_calculation(ori_path, pro_path, batch_size, num_workers):
  image_transforms = transforms.Compose([transforms.Resize((256,256)),
                                         transforms.ToTensor(),
                                         transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5]),
                                         transforms.CenterCrop(size=(112,112))])
  ori_dataset = LoadDataset(path=ori_path, transforms=image_transforms)
  pro_dataset = LoadDataset(path=pro_path, transforms=image_transforms)
  print("The number of input original images are {}, and processed images are {}".format(len(ori_dataset),len(pro_dataset)))

  ori_dataloader = DataLoader(
      dataset = ori_dataset,
      batch_size = batch_size,
      shuffle = False,
      num_workers = num_workers,
      drop_last = False
  )
  pro_dataloader = DataLoader(
      dataset = pro_dataset,
      batch_size = batch_size,
      shuffle = False,
      num_workers = num_workers,
      drop_last = False
  )
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  
  print("Running on {}\n".format(device))
  
  loss_func = torch.nn.MSELoss()
  loss_func.to(device)
  
  MSE_scores = []
  try:
    with tqdm(zip(ori_dataloader, pro_dataloader), total=min(len(ori_dataloader),len(pro_dataloader))) as t:
      for batch in t:
        score = float(loss_func(batch[0].to(device), batch[1].to(device)))
        MSE_scores.append(score)
  except KeyboardInterrupt:
    t.close()
    raise
  
  mse_mean = np.mean(MSE_scores)
  mse_std = np.std(MSE_scores)
  print("\n#MSE is {:.2f}+-{:.2f}".format(mse_mean,mse_std))

  return mse_mean, mse_std


# DeepPrivacy

*   The MSE value between original and DeepPrivacy Output Images is: 0.09+-0.04

In [ ]:
# Calculating MSE value between original images with CIAGAN output images
original_path = "/content/drive/MyDrive/Image_datasets/celeba_align_20k_21k_aligned"
processed_path = "./image_datasets/DeepPrivacy_celeba_20k_21k_output/generated"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#The MSE value between original and DeepPrivacy Output Images is: 


The number of input original images are 976, and processed images are 976
Running on cuda:0



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 244/244 [01:22<00:00,  2.95it/s]


#MSE is 0.09+-0.04


# CIAGAN

*   The MSE value between original and CIAGAN Output Images is: 0.07+-0.02



In [ ]:
# Calculating MSE value between original images with CIAGAN output images
original_path = "image_datasets/CIAGAN_celeba_20k_21k_output/original"
processed_path = "image_datasets/CIAGAN_celeba_20k_21k_output/generated"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#The MSE value between original and CIAGAN Output Images is: 0.07+-0.02


The number of input original images are 976, and processed images are 976
Running on cuda:0



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 244/244 [01:59<00:00,  2.04it/s]


#MSE is 0.07+-0.02


# Inferenced CelebA
###Calculating MSE value between aligned images with inferenced output images

* Privacy 0.0 MSE is 
* Privacy 0.1 MSE is 
* Privacy 0.2 MSE is 0.02+-0.00
* Privacy 0.3 MSE is 
* Privacy 0.4 MSE is 
* Privacy 0.5 MSE is 
* Privacy 0.6 MSE is 
* Privacy 0.7 MSE is 
* Privacy 0.8 MSE is 
* Privacy 0.9 MSE is 
* Privacy 1.0 MSE is 

In [6]:
# Calculating MSE value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.0/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.0/generated"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)


The number of input original images are 976, and processed images are 976
Running on cuda:0



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 244/244 [02:31<00:00,  1.61it/s]


#MSE is 0.02+-0.00


In [ ]:
# Calculating MSE value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.1/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.1/generated"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)


The number of input original images are 270, and processed images are 270
Running on cuda:0



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
 38%|███▊      | 26/68 [00:16<00:27,  1.51it/s]

In [ ]:
# Calculating MSE value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.2/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.2/generated"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)


The number of input original images are 976, and processed images are 976
Running on cuda:0



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 244/244 [00:58<00:00,  4.14it/s]


#MSE is 0.02+-0.00


In [ ]:
# Calculating MSE value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.3/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.3/generated"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)


In [ ]:
# Calculating MSE value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.4/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.4/generated"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)


In [ ]:
# Calculating MSE value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.5/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.5/generated"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)


In [ ]:
# Calculating MSE value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.6/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.6/generated"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)


In [ ]:
# Calculating MSE value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.7/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.7/generated"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)


In [ ]:
# Calculating MSE value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.8/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.8/generated"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)


In [ ]:
# Calculating MSE value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.9/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.9/generated"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)


In [ ]:
# Calculating MSE value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_1.0/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_1.0/generated"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)


# Inferenced
###Calculating MSE value between aligned images with inferenced output images

* Privacy 0.0 MSE is 0.04+-0.01
* Privacy 0.1 MSE is 0.04+-0.01
* Privacy 0.2 MSE is 0.03+-0.01
* Privacy 0.3 MSE is 0.03+-0.01
* Privacy 0.4 MSE is 0.03+-0.01
* Privacy 0.5 MSE is 0.03+-0.01
* Privacy 0.6 MSE is 0.03+-0.01
* Privacy 0.7 MSE is 0.03+-0.01
* Privacy 0.8 MSE is 0.03+-0.01
* Privacy 0.9 MSE is 0.03+-0.01
* Privacy 1.0 MSE is 0.03+-0.01

In [ ]:
"""# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.0"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.0 MSE is 0.05+-0.01"""
# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.0"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.0 MSE is 0.04+-0.01

The number of input original images are 998, and processed images are 998
Running on cuda:0



100%|██████████| 250/250 [04:11<00:00,  1.01s/it]


#MSE is 0.04+-0.01


In [ ]:
"""# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.1"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.1 MSE is 0.05+-0.01"""
# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.1"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.1 MSE is 0.04+-0.01

The number of input original images are 998, and processed images are 998
Running on cuda:0



100%|██████████| 250/250 [02:06<00:00,  1.97it/s]


#MSE is 0.04+-0.01


In [ ]:
"""# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.2"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.2 MSE is 0.04+-0.01"""
# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.2"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.2 MSE is 0.03+-0.01

The number of input original images are 998, and processed images are 998
Running on cuda:0



100%|██████████| 250/250 [02:16<00:00,  1.84it/s]


#MSE is 0.03+-0.01


In [ ]:
"""# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.3"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.3 MSE is 0.04+-0.01"""
# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.3"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.3 MSE is 0.03+-0.01

The number of input original images are 998, and processed images are 998
Running on cuda:0



100%|██████████| 250/250 [02:09<00:00,  1.94it/s]


#MSE is 0.03+-0.01


In [ ]:
"""# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.4"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.4 MSE is 0.04+-0.01"""
# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.4"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.4 MSE is 0.03+-0.01

The number of input original images are 998, and processed images are 998
Running on cuda:0



100%|██████████| 250/250 [02:09<00:00,  1.94it/s]


#MSE is 0.03+-0.01


In [ ]:
"""# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.5"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.5 MSE is 0.04+-0.01"""
# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.5"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.5 MSE is 0.03+-0.01

The number of input original images are 998, and processed images are 998
Running on cuda:0



100%|██████████| 250/250 [02:08<00:00,  1.95it/s]


#MSE is 0.03+-0.01


In [ ]:
"""# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.6"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.6 MSE is 0.04+-0.01"""
# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.6"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.6 MSE is 0.03+-0.01

The number of input original images are 998, and processed images are 998
Running on cuda:0



100%|██████████| 250/250 [02:11<00:00,  1.90it/s]


#MSE is 0.03+-0.01


In [ ]:
"""# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.7"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.7 MSE is 0.04+-0.01"""
# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.7"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.7 MSE is 0.03+-0.01

The number of input original images are 998, and processed images are 998
Running on cuda:0



100%|██████████| 250/250 [02:10<00:00,  1.92it/s]


#MSE is 0.03+-0.01


In [ ]:
"""# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.8"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.8 MSE is 0.04+-0.01"""
# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.8"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.8 MSE is 0.03+-0.01

The number of input original images are 998, and processed images are 998
Running on cuda:0



100%|██████████| 250/250 [02:09<00:00,  1.93it/s]


#MSE is 0.03+-0.01


In [ ]:
"""# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.9"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.9 MSE is 0.04+-0.01"""
# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.9"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.9 MSE is 0.03+-0.01

The number of input original images are 998, and processed images are 998
Running on cuda:0



100%|██████████| 250/250 [03:54<00:00,  1.07it/s]


#MSE is 0.03+-0.01


In [ ]:
"""# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_1.0"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 1.0 MSE is 0.04+-0.01"""
# Calculating MSE value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_1.0"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 1.0 MSE is 0.03+-0.01

The number of input original images are 998, and processed images are 998
Running on cuda:0



100%|██████████| 250/250 [03:37<00:00,  1.15it/s]


#MSE is 0.03+-0.01


In [ ]:
### ffhq10k_11k ###
#Privacy 0.0 MSE is 0.04+-0.01
#Privacy 0.1 MSE is 0.04+-0.01
#Privacy 0.2 MSE is 0.03+-0.01
#Privacy 0.3 MSE is 0.03+-0.01
#Privacy 0.4 MSE is 0.03+-0.01
#Privacy 0.5 MSE is 0.03+-0.01
#Privacy 0.6 MSE is 0.03+-0.01
#Privacy 0.7 MSE is 0.03+-0.01
#Privacy 0.8 MSE is 0.03+-0.01
#Privacy 0.9 MSE is 0.03+-0.01
#Privacy 1.0 MSE is 0.03+-0.01

### ffhq5k_6k ###
#Privacy 0.0 MSE is 0.04+-0.01
#Privacy 0.1 MSE is 0.04+-0.01
#Privacy 0.2 MSE is 0.04+-0.01
#Privacy 0.3 MSE is 0.04+-0.01
#Privacy 0.4 MSE is 0.04+-0.01
#Privacy 0.5 MSE is 0.04+-0.01
#Privacy 0.6 MSE is 0.04+-0.01
#Privacy 0.7 MSE is 0.04+-0.01
#Privacy 0.8 MSE is 0.04+-0.01
#Privacy 0.9 MSE is 0.04+-0.01
#Privacy 1.0 MSE is 0.04+-0.01

# Blurred

In [ ]:
"""# Calculating MSE value between aligned images with blurred output images
original_path = "image_datasets/ffhq5k_6k_blurred_ksize_349/ffhq5k_6k_blurred_ori"
processed_path = "image_datasets/ffhq5k_6k_blurred_ksize_349/ffhq5k_6k_blurred"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size 349 MSE is 0.12+-0.03"""
# Calculating MSE value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_349/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_349/ffhq10k_11k_blurred"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size 349 MSE is 0.12+-0.03

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [04:06<00:00,  1.00it/s]


#MSE is 0.12+-0.03


In [ ]:
"""# Calculating MSE value between aligned images with blurred output images
original_path = "image_datasets/ffhq5k_6k_blurred_ksize_149/ffhq5k_6k_blurred_ori"
processed_path = "image_datasets/ffhq5k_6k_blurred_ksize_149/ffhq5k_6k_blurred"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size 149 MSE is 0.10+-0.02"""


In [ ]:
"""# Calculating MSE value between aligned images with blurred output images
original_path = "image_datasets/ffhq5k_6k_blurred_ksize_099/ffhq5k_6k_blurred_ori"
processed_path = "image_datasets/ffhq5k_6k_blurred_ksize_099/ffhq5k_6k_blurred"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size 099 MSE is 0.06+-0.01"""


In [ ]:
"""# Calculating MSE value between aligned images with blurred output images
original_path = "image_datasets/ffhq5k_6k_blurred_ksize_065/ffhq5k_6k_blurred_ori"
processed_path = "image_datasets/ffhq5k_6k_blurred_ksize_065/ffhq5k_6k_blurred"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size 065 MSE is 0.05+-0.01"""
# Calculating MSE value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_065/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_065/ffhq10k_11k_blurred"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size 065 MSE is 0.05+-0.01

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [02:03<00:00,  2.00it/s]


#MSE is 0.05+-0.01


In [ ]:
# Calculating MSE value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_059/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_059/ffhq10k_11k_blurred"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size 059 MSE is 0.04+-0.01

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [02:07<00:00,  1.93it/s]


#MSE is 0.04+-0.01


In [ ]:
"""# Calculating MSE value between aligned images with blurred output images
original_path = "image_datasets/ffhq5k_6k_blurred_ksize_049/ffhq5k_6k_blurred_ori"
processed_path = "image_datasets/ffhq5k_6k_blurred_ksize_049/ffhq5k_6k_blurred"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size 049 MSE is 0.03+-0.01"""


In [ ]:
# Calculating MSE value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_043/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_043/ffhq10k_11k_blurred"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size 043 MSE is 0.03+-0.01

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [02:14<00:00,  1.84it/s]


#MSE is 0.03+-0.01


In [ ]:
"""# Calculating MSE value between aligned images with blurred output images
original_path = "image_datasets/ffhq5k_6k_blurred_ksize_039/ffhq5k_6k_blurred_ori"
processed_path = "image_datasets/ffhq5k_6k_blurred_ksize_039/ffhq5k_6k_blurred"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size 039 MSE is 0.03+-0.01"""
# Calculating MSE value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_039/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_039/ffhq10k_11k_blurred"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size 039 MSE is 0.03+-0.01

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [02:07<00:00,  1.93it/s]


#MSE is 0.03+-0.01


In [ ]:
# Calculating MSE value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_031/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_031/ffhq10k_11k_blurred"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size 031 MSE is 

In [ ]:
"""# Calculating MSE value between aligned images with blurred output images
original_path = "image_datasets/ffhq5k_6k_blurred_ksize_029/ffhq5k_6k_blurred_ori"
processed_path = "image_datasets/ffhq5k_6k_blurred_ksize_029/ffhq5k_6k_blurred"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size 029 MSE is 0.02+-0.00"""

In [ ]:
"""# Calculating MSE value between aligned images with blurred output images
original_path = "image_datasets/ffhq5k_6k_blurred_ksize_009/ffhq5k_6k_blurred_ori"
processed_path = "image_datasets/ffhq5k_6k_blurred_ksize_009/ffhq5k_6k_blurred"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size 009 MSE is 0.01+-0.00"""

# Mosaiced

In [ ]:
# Calculating MSE value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor100/ffhq10k_11k_mosaiced_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor100/ffhq10k_11k_mosaiced"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor 100 MSE is 0.42+-0.13

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [03:20<00:00,  1.23it/s]


#MSE is 0.42+-0.13


In [ ]:
# Calculating MSE value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor80/ffhq10k_11k_mosaiced_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor80/ffhq10k_11k_mosaiced"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor 80 MSE is 0.35+-0.10

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [03:13<00:00,  1.27it/s]


#MSE is 0.35+-0.10


In [ ]:
"""# Calculating MSE value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor50/ffhq5k_6k_mosaiced_ori"
processed_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor50/ffhq5k_6k_mosaiced"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor 50 MSE is 0.26+-0.08"""


In [ ]:
"""# Calculating MSE value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor25/ffhq5k_6k_mosaiced_ori"
processed_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor25/ffhq5k_6k_mosaiced"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor 25 MSE is 0.14+-0.04"""


In [ ]:
"""# Calculating MSE value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor20/ffhq5k_6k_mosaiced_ori"
processed_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor20/ffhq5k_6k_mosaiced"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor 20 MSE is 0.15+-0.04"""


In [ ]:
"""# Calculating MSE value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor15/ffhq5k_6k_mosaiced_ori"
processed_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor15/ffhq5k_6k_mosaiced"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor 15 MSE is 0.11+-0.03"""


In [ ]:
"""# Calculating MSE value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor13/ffhq5k_6k_mosaiced_ori"
processed_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor13/ffhq5k_6k_mosaiced"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor 13 MSE is 0.10+-0.02"""
# Calculating MSE value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor13/ffhq10k_11k_mosaiced_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor13/ffhq10k_11k_mosaiced"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor 13 MSE is 0.10+-0.02

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [02:02<00:00,  2.01it/s]


#MSE is 0.10+-0.02


In [ ]:
"""# Calculating MSE value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor10/ffhq5k_6k_mosaiced_ori"
processed_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor10/ffhq5k_6k_mosaiced"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor 10 MSE is 0.08+-0.02"""


In [ ]:
"""# Calculating MSE value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor09/ffhq5k_6k_mosaiced_ori"
processed_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor09/ffhq5k_6k_mosaiced"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor 09 MSE is 0.07+-0.02"""
# Calculating MSE value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor09/ffhq10k_11k_mosaiced_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor09/ffhq10k_11k_mosaiced"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor 09 MSE is 0.07+-0.02

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [02:14<00:00,  1.84it/s]


#MSE is 0.07+-0.02


In [ ]:
# Calculating MSE value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor07/ffhq10k_11k_mosaiced_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor07/ffhq10k_11k_mosaiced"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor 07 MSE is 0.05+-0.01

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [02:03<00:00,  2.01it/s]


#MSE is 0.05+-0.01


In [ ]:
"""# Calculating MSE value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor05/ffhq5k_6k_mosaiced_ori"
processed_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor05/ffhq5k_6k_mosaiced"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor 05 MSE is 0.03+-0.01"""


In [ ]:
# Calculating MSE value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor04/ffhq10k_11k_mosaiced_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor04/ffhq10k_11k_mosaiced"
mse_mean, mse_std = MSE_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor 04 MSE is 0.03+-0.01

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [02:01<00:00,  2.03it/s]


#MSE is 0.03+-0.01
